# Import libraries

In [1]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualization
import seaborn as sns # visualization
from pyarabic.araby import tokenize # tokenize text data
from sklearn.model_selection import train_test_split #split data
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score
import sqlite3
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# modfy size of row and column for pandas data frame
pd.options.display.max_rows = 200 
pd.options.display.max_columns = 1900
pd.set_option('display.max_colwidth', 1000)

# load data

In [5]:
data = pd.read_csv("/kaggle/input/cleaned-dataset/cleaned_data.csv")
data = data.drop(columns=["Unnamed: 0"])

In [6]:
data.head(5)

,id,dialect,text
0,1009754958479151232,LY,قليلين ادب ومنافقين اختهم قريبتهم تتعاكس تقولي عليهم نشاط حقوق المرءة ردة فعلهم
1,1009794751548313600,LY,الليبيين متقلبين بالنسبة ليا ميليشياوي زمان وتوة
2,1019989115490787200,LY,تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها البنات وبيحس كءنه يعرفها زمان بعدين يتزوج وحدة منهن وممكن ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب اند
3,1035479791758135168,LY,رانيا عقليتك متخلفة اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم انسان قليل الادب ثانيا شن ذنب يلي معندهش اب خوت خوات يعني اليتيمة متستحقش تتزوج وثالثا ليش البنت لازم ادير حساب للراجل متستحقش يندارلها حساب عبدة
4,1035481122921164800,LY,شكلك متعقدة علشان الراجل تحبيه ازوج بنت يتيمة بنت معندهش خوت هدي اعصابك وفكينا التخلف امتاعك


# Tokenize text

In [7]:
def araby_tokenizer(sentence):
    tokens = tokenize(sentence)
    return tokens

In [8]:
data["tokenize_data"] = data["text"].apply(araby_tokenizer)

In [9]:
data.head(5)

,id,dialect,text,tokenize_data
0,1009754958479151232,LY,قليلين ادب ومنافقين اختهم قريبتهم تتعاكس تقولي عليهم نشاط حقوق المرءة ردة فعلهم,"[قليلين, ادب, ومنافقين, اختهم, قريبتهم, تتعاكس, تقولي, عليهم, نشاط, حقوق, المرءة, ردة, فعلهم]"
1,1009794751548313600,LY,الليبيين متقلبين بالنسبة ليا ميليشياوي زمان وتوة,"[الليبيين, متقلبين, بالنسبة, ليا, ميليشياوي, زمان, وتوة]"
2,1019989115490787200,LY,تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها البنات وبيحس كءنه يعرفها زمان بعدين يتزوج وحدة منهن وممكن ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب اند,"[تانيه, شاب, ليبي, بيرتاح, لبنت, مختلفة, ويلاحظ, انها, البنات, وبيحس, كءنه, يعرفها, زمان, بعدين, يتزوج, وحدة, منهن, وممكن, ولاثلاثة, وتنقلب, الرومانسية, لعياط, وشياط, وتهزيب, اند]"
3,1035479791758135168,LY,رانيا عقليتك متخلفة اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم انسان قليل الادب ثانيا شن ذنب يلي معندهش اب خوت خوات يعني اليتيمة متستحقش تتزوج وثالثا ليش البنت لازم ادير حساب للراجل متستحقش يندارلها حساب عبدة,"[رانيا, عقليتك, متخلفة, اولا, الانسان, يلي, يحتاج, اهل, يخاف, منهم, علشان, يكون, محترم, انسان, قليل, الادب, ثانيا, شن, ذنب, يلي, معندهش, اب, خوت, خوات, يعني, اليتيمة, متستحقش, تتزوج, وثالثا, ليش, البنت, لازم, ادير, حساب, للراجل, متستحقش, يندارلها, حساب, عبدة]"
4,1035481122921164800,LY,شكلك متعقدة علشان الراجل تحبيه ازوج بنت يتيمة بنت معندهش خوت هدي اعصابك وفكينا التخلف امتاعك,"[شكلك, متعقدة, علشان, الراجل, تحبيه, ازوج, بنت, يتيمة, بنت, معندهش, خوت, هدي, اعصابك, وفكينا, التخلف, امتاعك]"


# Get word to vec for each word and then get mean of them to construct text vector

In [10]:
# Download model 
!wget "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_sg_300_twitter.zip"
!unzip "/kaggle/working/full_uni_sg_300_twitter.zip"


--2023-05-21 21:01:42--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_sg_300_twitter.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2832094149 (2.6G) [application/zip]
Saving to: ‘full_uni_sg_300_twitter.zip’

full_uni_sg_300_twi 100%[===================>]   2.64G   124MB/s    in 32s     

2023-05-21 21:02:14 (85.2 MB/s) - ‘full_uni_sg_300_twitter.zip’ saved [2832094149/2832094149]

Archive:  /kaggle/working/full_uni_sg_300_twitter.zip
  inflating: full_uni_sg_300_twitter.mdl  
  inflating: full_uni_sg_300_twitter.mdl.trainables.syn1neg.npy  
  inflating: full_uni_sg_300_twitter.mdl.wv.vectors.npy  


In [11]:
t_model = gensim.models.Word2Vec.load('/kaggle/working/full_uni_sg_300_twitter.mdl')

In [12]:
def get_word_vec_for_each_text(text):
    vector_size = t_model.wv.vector_size
    word_to_vec_res = np.zeros(vector_size)
    # count number of found words 
    number_of_found_words = 1
    for word in text:
        if word in t_model.wv:
            number_of_found_words += 1
            word_to_vec_res += t_model.wv[word]
    # get mean of each word vector
    word_to_vec_res = word_to_vec_res/number_of_found_words
    return word_to_vec_res

In [13]:
data["vector"] = data["tokenize_data"].apply(get_word_vec_for_each_text)

In [14]:
data.head(1)

,id,dialect,text,tokenize_data,vector
0,1009754958479151232,LY,قليلين ادب ومنافقين اختهم قريبتهم تتعاكس تقولي عليهم نشاط حقوق المرءة ردة فعلهم,"[قليلين, ادب, ومنافقين, اختهم, قريبتهم, تتعاكس, تقولي, عليهم, نشاط, حقوق, المرءة, ردة, فعلهم]","[0.1415936080738902, 0.00507965338571618, 0.03601026154744128, 0.010379871974388758, -0.030205688565426197, 0.14799912553280592, 0.0785125185114642, 0.2115561767326047, 0.023974398771921795, -0.15613386411375055, 0.11379470024257898, 0.07379899515459935, 0.20598448098947605, 0.012382933404296637, 0.1682413968568047, -0.09552373566354315, -0.18372443256278834, -0.031194643427928288, 0.06802133998523156, -0.22994574718662383, 0.07677711406722665, -0.11481489893049002, -0.20459687833984694, -0.07121558184735477, -0.13083617885907492, -0.13210957761233053, 0.23462902723501125, 0.058982972792970635, -0.09097434083620708, 0.22893495004003248, 0.048678600850204624, -0.012876965609999994, -0.1083035843524461, -0.04163527116179466, -0.02476840854311983, 0.18855790871505937, 0.15565243549644947, 0.19553360405067602, -0.0009464264536897341, 0.10604396679749091, -0.060508602880872786, 0.03452180733438581, -0.017178657581098378, 0.01956355633834998, -0.18149941693991423, -0.0614716246103247, 0...."


# Split data

In [15]:
X = data['vector'].values
X = (np.array([X[i] for i in range(len(X))]))
y = data['dialect'].values

In [16]:
labels = list(data['dialect'].unique())

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [18]:
clf1 = LogisticRegression(max_iter=1000)
clf1.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [19]:
print(classification_report(y_test, clf1.predict(X_test), target_names=labels))

              precision    recall  f1-score   support

          LY       0.82      0.91      0.86     11527
          MA       0.82      0.79      0.81      5523
          EG       0.72      0.77      0.74      7300
          LB       0.80      0.60      0.69      2308
          SD       0.68      0.46      0.54      2887

    accuracy                           0.78     29545
   macro avg       0.77      0.70      0.73     29545
weighted avg       0.78      0.78      0.78     29545



In [20]:
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(X_train, y_train)

LGBMClassifier()

In [21]:
print(classification_report(y_test, lgb_clf.predict(X_test), target_names=labels))

              precision    recall  f1-score   support

          LY       0.79      0.91      0.85     11527
          MA       0.82      0.75      0.78      5523
          EG       0.67      0.78      0.72      7300
          LB       0.86      0.53      0.66      2308
          SD       0.74      0.36      0.48      2887

    accuracy                           0.76     29545
   macro avg       0.78      0.67      0.70     29545
weighted avg       0.77      0.76      0.75     29545



# Try text word_vectors with CNN instead of calculate mean 

In [ ]:
x = data.filter(items=["tokenize_data","id"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, data["dialect"], test_size=0.1, random_state=42)

In [ ]:
def get_longest_text(texts):
    longest_input = 0
    for text in texts:
        text_len= len(text.split())
        longest_input = max(longest_input, text_len)
    return longest_input

In [ ]:
longest_input = get_longest_text(data['text'])

In [ ]:
vector_size = t_model.wv.vector_size
data_emb_train = np.zeros((len(X_train), longest_input+1 , 300))
for i, text in enumerate(X_train["tokenize_data"]):
    for j, word in enumerate(text):
        if word in t_model.wv:
            data_emb_train[i, j] = t_model.wv[word]


In [ ]:
vector_size = t_model.wv.vector_size
data_emb_test = np.zeros((len(X_test), longest_input+1 , 300))
for i, text in enumerate(X_test["tokenize_data"]):
    for j, word in enumerate(text):
        if word in t_model.wv:
            data_emb_test[i, j] = t_model.wv[word]

In [ ]:
data_emb_train.shape

In [ ]:
y_train.shape

In [ ]:
data_emb_test.shape

In [ ]:
import tensorflow as tf

# define the network
inputs = tf.keras.layers.Input((longest_input+1, 300))
reshaped = tf.keras.layers.Reshape((longest_input+1, 300, 1))(inputs)


filters = [2, 3, 4]

# define the conv net
conv_1 = tf.keras.layers.Conv2D(100, (filters[0], 300), activation='relu')(reshaped)
conv_2 = tf.keras.layers.Conv2D(100, (filters[1], 300), activation='relu')(reshaped)
conv_3 = tf.keras.layers.Conv2D(100, (filters[2], 300), activation='relu')(reshaped)

# define max-pooling
pool_1 = tf.keras.layers.MaxPooling2D((longest_input - filters[0] + 1, 1), strides=(1,1))(conv_1)
pool_2 = tf.keras.layers.MaxPooling2D((longest_input - filters[1] + 1, 1), strides=(1,1))(conv_2)
pool_3 = tf.keras.layers.MaxPooling2D((longest_input - filters[2] + 1, 1), strides=(1,1))(conv_3)

# concatenate the convs
merged_tensor = tf.keras.layers.concatenate([pool_1, pool_2, pool_3], axis=1)

# now flatten them and add a dense layer
flatten = tf.keras.layers.Flatten()(merged_tensor)

# add a dense layer
clf = tf.keras.layers.Dense(100, activation='relu')(flatten)

# add final output
clf = tf.keras.layers.Dense(len(labels), activation='softmax')(clf)

In [ ]:
model = tf.keras.models.Model(inputs, clf)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['acc'])

In [ ]:
#model.fit(data_emb_train, y_train, epochs=23)

In [ ]:
print(classification_report(y_test, np.argmax(model.predict(data_emb_test),axis=1), target_names=labels))

# Try Tfidf

In [23]:
le = LabelEncoder()
data['dialect'] = le.fit_transform(data['dialect'])
data['text'] = data['text'].fillna('')

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
        data['text'], data['dialect'], random_state=42, test_size=0.1, shuffle=True)

In [25]:
tfidf = TfidfVectorizer(
    max_df=0.95, 
    ngram_range=(1, 5),
    max_features= 25000,
    analyzer='word')

In [26]:
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [27]:
lr = LogisticRegression(multi_class='ovr')
lr.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='ovr')

In [28]:
y_pred = lr.predict(X_test)
accuracy =accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("Model accuracy:", accuracy)
print("Classification report:\n",report)

Model accuracy: 0.8111419481486496
Classification report:
               precision    recall  f1-score   support

           0       0.81      0.92      0.86      5779
           1       0.86      0.82      0.84      2790
           2       0.75      0.81      0.78      3549
           3       0.93      0.60      0.73      1169
           4       0.84      0.52      0.64      1486

    accuracy                           0.81     14773
   macro avg       0.84      0.74      0.77     14773
weighted avg       0.82      0.81      0.81     14773



In [29]:
lgb_clf = lgb.LGBMClassifier()
lgb_clf.fit(X_train, y_train)

LGBMClassifier()

In [30]:
y_pred = lgb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("Modelaccuracy:", accuracy)
print("Classification report:\n", report)

Modelaccuracy: 0.7493400121843904
Classification report:
               precision    recall  f1-score   support

           0       0.80      0.85      0.83      5779
           1       0.87      0.71      0.78      2790
           2       0.59      0.80      0.68      3549
           3       0.91      0.55      0.68      1169
           4       0.81      0.47      0.59      1486

    accuracy                           0.75     14773
   macro avg       0.80      0.67      0.71     14773
weighted avg       0.77      0.75      0.75     14773



# Save best model logistic regression with tfidf --> 77 macro

In [31]:
from joblib import dump, load

In [32]:
dump(lr, 'logistic_regression_model.joblib')

['logistic_regression_model.joblib']

# Note

In [33]:
# load the model from the file
loaded_model = load('/kaggle/working/logistic_regression_model.joblib')
loaded_model.predict(X_test[0])

array([3])

In [34]:
loaded_model.classes_

array([0, 1, 2, 3, 4])